In [1]:
from DICOMLib import DICOMUtils
import os 
import vtkITK
import vtkSegmentationCorePython as vtkSegmentationCore
import vtkSlicerSegmentationsModuleLogicPython as vtkSlicerSegmentationsModuleLogic
import qt


# Set paths to the input and output directory
dicomFilesDirectory = 'C:/dev/data/CT_Head/raw_dicoms/'
outputDirectory = "C:/dev/data/CT_Head/stripped_dicoms/"

# Note: you can play around with the following variables, see what might work better for your data:
#    - smoothing kernel size (set to 3)
#    - minimum threshold (set to 200 HU)
#    - split cavaties diameter (set to 30 mm)


# *************************************
# ***** IMPORT DICOMS IN DATABASE *****
# *************************************

settings = qt.QSettings()
settings.setValue("DICOM/ScalarVolume/AcquisitionGeometryRegularization", "transform")
print("settings...")

file_ids = [x for x in os.listdir(dicomFilesDirectory)]    
print("got ids...")

DICOMUtils.clearDatabase()
DICOMUtils.importDicom(dicomFilesDirectory)
print("imported...")
      
# slicer's dicom database containing imported dicoms
db = slicer.dicomDatabase
# get List of patient UIDs of dicoms
#patientUIDs = db.patients()
#print(len(patientUIDs))

seriesUIDs = DICOMUtils.allSeriesUIDsInDatabase(db)
print(len(seriesUIDs))


# ***************************
# ***** FOR EVERY DICOM *****
# ***************************

for idx, uid in enumerate(seriesUIDs):
    
    # *********************
    # ***** LOAD DATA *****
    # *********************
    
    print('processing patient: ' + uid)
    loadedNodeIDs = DICOMUtils.loadSeriesByUID([uid]) # get ID of loaded data 
    print('loading succesful: ' + str(len(loadedNodeIDs)))

    masterVolumeNode = slicer.util.getNode(loadedNodeIDs[0]) # get volume by ID
    masterVolumeNode.HardenTransform()
    smoothingKernelSizeMm = 3.0  # this is used for closing small holes in the segmentation
    
    # Create segment editor (get access to effects)
    slicer.app.processEvents()
    segmentEditorWidget = slicer.qMRMLSegmentEditorWidget()    # Create new segmentEditorWidge
    segmentEditorWidget.setMRMLScene(slicer.mrmlScene)    # Add EditorWidget to scene
    segmentEditorWidget.show()    # Display editor widget

    # Create segmentEditorNode
    segmentEditorNode = slicer.vtkMRMLSegmentEditorNode()
    slicer.mrmlScene.AddNode(segmentEditorNode)

    segmentEditorWidget.setMasterVolumeNode(masterVolumeNode)    # masterVolume
    segmentEditorWidget.setMRMLSegmentEditorNode(segmentEditorNode)    # segmentEditorNode

    
    # ************************************
    # ***** CREATE SEGMENTATION NODE *****
    # ************************************

    # Create new SegmentationNode and pass masterVolume
    print('Create segmentation Node and pass master Volume')
    slicer.app.processEvents()
    segmentationNode = slicer.vtkMRMLSegmentationNode()    # Create new segmentation node
    slicer.mrmlScene.AddNode(segmentationNode)    # Add new node to scene
    segmentationNode.CreateDefaultDisplayNodes()    # only needed for display
    segmentationNode.SetReferenceImageGeometryParameterFromVolumeNode(masterVolumeNode)    # Pass master volume as reference

    # Connect everything
    segmentEditorWidget.setSegmentationNode(segmentationNode)    # segmentationNode
    
    
    # *************************
    # ***** APPLY EFFECTS *****
    # *************************

    # Get pixel value range
    volumeScalarRange = masterVolumeNode.GetImageData().GetScalarRange()
    slicer.app.processEvents()
    
    # SEGMENT 1: SKULL

    # Create bone segment by thresholding
    slicer.app.processEvents()
    skullSegmentID = segmentationNode.GetSegmentation().AddEmptySegment("skull")
    segmentEditorNode.SetSelectedSegmentID(skullSegmentID)
    segmentEditorWidget.setActiveEffectByName("Threshold")
    effect = segmentEditorWidget.activeEffect()
    effect.setParameter("MinimumThreshold",str(200))
    effect.setParameter("MaximumThreshold",str(volumeScalarRange[1]))
    effect.self().onApply()
    print('1')

    # hilft vielleicht:
    # Smooth bone segment (just to reduce solidification computation time)
    #slicer.app.processEvents()
    #segmentEditorWidget.setActiveEffectByName("Smoothing")
    #effect = segmentEditorWidget.activeEffect()
    #effect.setParameter("SmoothingMethod", "MORPHOLOGICAL_CLOSING")
    #effect.setParameter("KernelSizeMm", str(3))
    #effect.self().onApply()
    #print('2')
    
    # Find largest cavity
    #slicer.app.processEvents()
    #segmentEditorWidget.setActiveEffectByName("Islands")
    #effect = segmentEditorWidget.activeEffect()
    #effect.setParameterDefault("Operation", "KEEP_LARGEST_ISLAND")
    #effect.self().onApply()
    #print('4')

    # Solidify bone
    slicer.app.processEvents()
    segmentEditorWidget.setActiveEffectByName("Wrap Solidify")
    effect = segmentEditorWidget.activeEffect()
    effect.setParameter("region", "largestCavity")
    effect.setParameter("splitCavities", "True")
    effect.setParameter("splitCavitiesDiameter", "30.0")
    effect.setParameter("outputType", "newSegment")
    effect.setParameter("remeshOversampling", "1.0")
    segmentEditorWidget.setCurrentSegmentID(segmentationNode.GetSegmentation().GetSegmentIdBySegmentName('skull'))
    effect.self().onApply()
    print('5')
    
        # hilft vielleicht:
    # Smooth bone segment (just to reduce solidification computation time)
    #slicer.app.processEvents()
    #segmentEditorWidget.setActiveEffectByName("Smoothing")
    #effect = segmentEditorWidget.activeEffect()
    #effect.setParameter("SmoothingMethod", "MORPHOLOGICAL_CLOSING")
    #effect.setParameter("KernelSizeMm", str(3))
    #effect.self().onApply()
    #print('6')
    
    # Blank out the volume outside the object segment
    slicer.app.processEvents()
    segmentEditorWidget.setActiveEffectByName('Mask volume')
    effect = segmentEditorWidget.activeEffect()
    segmentEditorNode.SetSelectedSegmentID(segmentationNode.GetSegmentation().GetSegmentIdBySegmentName('skull_solid'))
    effect.setParameter('FillValue', -1024)
    effect.setParameter('Operation', 'FILL_OUTSIDE')
    effect.self().onApply()
    print('5')

    # Show masked volume
    maskedVolume = slicer.mrmlScene.GetFirstNodeByName(masterVolumeNode.GetName()+" masked")
    slicer.util.setSliceViewerLayers(background=maskedVolume)
    
    # Save Segmentation
    myStorageNode = maskedVolume.CreateDefaultStorageNode()
    path = os.path.join(outputDirectory, str(file_ids[idx]) + "_masked.nrrd")
    slicer.util.saveNode(maskedVolume, path)

    #myStorageNode.SetFileName(path)
    #print(path)
    #myStorageNode.WriteData(maskedVolume)
    
    # remove nodes
    slicer.mrmlScene.RemoveNode(maskedVolume)
    slicer.mrmlScene.RemoveNode(masterVolumeNode)
    slicer.mrmlScene.RemoveNode(segmentationNode)


SyntaxError: invalid syntax (3627058317.py, line 1)